# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5NDc0ZmNhNi0wODFlLTRhYTktYjgwZS01MWJkMDMxNWJhNTAifQ=="
os.environ['NEPTUNE_PROJECT']="guilhermemg/icao-nets-training-2"
os.environ['NEPTUNE_NOTEBOOK_ID']="98a391a1-c710-40bd-aaf4-42c31862cbbe"
os.environ['NEPTUNE_NOTEBOOK_PATH']="training/exec_nas_experiment.ipynb"

In [3]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0, '..')

In [4]:
from src.m_utils import constants as cts
from src.base.data_loaders.data_loader import DLName
from src.base.gt_loaders.gt_names import GTName
from src.exp_runner import ExperimentRunner

from src.base.experiment.dataset.dataset import Dataset
from src.base.experiment.evaluation.model_evaluator import DataSource, DataPredSelection
from src.base.experiment.training.base_models import BaseModel
from src.base.experiment.training.optimizers import Optimizer

from src.m_utils.stl_approach import STLApproach
from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.nas_mtl_approach import NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [5]:
DATASET = Dataset.MNIST
APPROACH = NAS_MTLApproach.APPROACH_3

kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'NAS experiment',
        'description': 'NAS with Approach 3',
        'tags': [f'{DATASET.value["name"]}', 'nas', 'nas_approach_3', 'no_use_predictor'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': APPROACH,
        'dataset': DATASET,
        'tasks': DATASET.value['tasks'],
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': True,
        'sample_prop': 0.03
    },
    'nas_params': {
        'architecture_training_epochs': 2,     # n_epochs for training proposed architecture
        'total_num_proposed_architectures': 6,
        'nas_algorithm': 'rl',
        'nas_search_space': 'ss_1'
    },
    'controller_params': {
        'controller_lstm_dim': 100,
        'controller_optimizer': Optimizer.ADAM,
        'controller_learning_rate': 0.01,
        'controller_decay': 0.1,
        'controller_momentum': 0.0,
        'controller_use_predictor': False,
        'controller_loss_alpha': 0.9,
        'controller_training_epochs': 5,
        'controller_sampling_epochs': 6,
        'controller_batch_size': 3
    },
    'mlp_params': {
        'max_architecture_length': 5,
        'min_task_group_size': 3,
        'mlp_base_model': BaseModel.MOBILENET_V2,
        'mlp_n_epochs': 50,
        'mlp_batch_size': 64,
        'mlp_early_stopping': 50,
        'mlp_optimizer': Optimizer.ADAMAX,
        'mlp_learning_rate': 1e-3,
        'mlp_decay': 0.0,
        'mlp_momentum': 0.0,
        'mlp_dropout': 0.3,
        'mlp_loss_function': 'sparse_categorical_crossentropy',
        'mlp_one_shot': True
    }
}


runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 17528
Process ID: 24256
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'controller_params': {'controller_batch_size': 3,
                       'controller_decay': 0.1,
                       'controller_learning_rate': 0.01,
                       'controller_loss_alpha': 0.9,
                       'controller_lstm_dim': 100,
                       'controller_momentum': 0.0,
                       'controller_optimizer': <Optimizer.ADAM: 'Adam'>,
                       'controller_sampling_epochs': 6,
                       'controller_training_epochs': 5,
                       'controller_use_predictor': False},
 'exp_params': {'description': 'NAS with Approach 3',
                'name': 'NAS experiment',
                'src_files': ['../src/**/*.py'],
                'tags': ['mnist', 'nas', 'nas_approach_3', 'no_use_predictor']},

# Load Data

In [6]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
TrainData.shape: (48000, 11)
ValidationData.shape: (12000, 11)
TestData.shape: (10000, 11)
Data loaded


In [7]:
runner.sample_training_data()

-------------------- sample training data -------------------
Applying subsampling in training data
..Sampling proportion: 0.03 (1440/48000)
(1440, 11)
Applying subsampling in validation data
..Sampling proportion: 0.03 (360/12000)
(360, 11)


In [8]:
runner.train_data.head()

,img_name,n_0,n_1,n_2,n_3,n_4,n_5,n_6,n_7,n_8,n_9
32771,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39512,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
43581,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2975,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3377,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Generators

In [9]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 1440 validated image filenames.
Found 360 validated image filenames.
Found 10000 validated image filenames.
TOTAL: 11800

Logging class indices
 .. MTL model not logging class indices!

Using benchmarking dataset. Not logging class labels!


# Setup Experiment

In [10]:
runner.setup_experiment()

-------------------- create experiment -------------------
Setting up neptune experiment
Neptune experiment setup done!


# Labels Distribution

In [11]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search - v3

In [12]:
best_arch = runner.run_neural_architecture_search_v3()

-------------------- run neural architecture search -------------------
all_archs[:10]: [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 2], [0, 0, 0, 3], [0, 0, 0, 4], [0, 0, 1, 0], [0, 0, 1, 1], [0, 0, 1, 2], [0, 0, 1, 3], [0, 0, 1, 4]]
search_space_size: 625
Controller model input shape: (None, 1, 4)
Controller model output shape: (None, 1, 625)
Model path:  trained_model
----
Checking model existence locally...
Training a new model! Not checking model existence
----
------------------------------
Checking GPU availability
 ..GPU is available!
------------------------------
 ..prev_arch: [[[4 2 5 1]]]
 ..prev_arch.shape: (1, 1, 4)
input: [[[4 2 5 1]]]
prob_list[:5]: [0.00152527 0.00149151 0.00164413 0.0016108  0.00160038]
chose_idx: 296
new_arch: [2, 1, 4, 1]
  New Controller Epoch | Feedback ID: 1 | Feedback DNA: DNA([2, 1, 4, 1])
----------------------------------------------------------------------
------------------------------------------------------------
 -- Architecture 1: {'n_denses_

/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '



Epoch 00002: val_loss improved from 0.51239 to 0.49686, saving model to training/training_ckpt/best_model.hdf5
..Loading best model
..Checkpoint weights loaded
Testing Trained Model
Predicting labels....
6/6 [==============================] - 1s 109ms/step
Prediction finished!
  Task  0: n_0             | EER_mean: 0.01230 | EER_interp: 0.00000 | ACC: 0.98890
  Task  1: n_1             | EER_mean: 0.00960 | EER_interp: 0.00000 | ACC: 0.99170
  Task  2: n_2             | EER_mean: 0.10280 | EER_interp: 0.11550 | ACC: 0.89440
  Task  3: n_3             | EER_mean: 0.12570 | EER_interp: 0.12060 | ACC: 0.87500
  Task  4: n_4             | EER_mean: 0.07890 | EER_interp: 0.07440 | ACC: 0.92220
  Task  5: n_5             | EER_mean: 0.09670 | EER_interp: 0.08290 | ACC: 0.90560
  Task  6: n_6             | EER_mean: 0.10000 | EER_interp: 0.10000 | ACC: 0.90000
  Task  7: n_7             | EER_mean: 0.10000 | EER_interp: 0.09240 | ACC: 0.91390
  Task  8: n_8             | EER_mean: 0.03720 | 

# Create Model with Best Architecture Found

In [14]:
runner.create_model(best_arch)

-------------------- create model -------------------
Creating model...
Model created


# Visualize Model

In [ ]:
%%capture
runner.visualize_model(outfile_path=f"training/figs/nas/nas_model_{APPROACH.name}.png")

In [ ]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()